# noxinfluencer(https://kr.noxinfluencer.com/) 크롤링

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from html_table_parser import parser_functions as parser
import requests
import json
import time
import os
import sys

In [2]:
name = '쯔양'
query_text = 'https://www.youtube.com/channel/UCfpaSruWW3S4dibonKXENjA' #input('크롤링할 유튜브 주소를 입력하세요 : ')

In [3]:
url = 'https://kr.noxinfluencer.com/youtube/' + query_text.split("https://www.youtube.com/")[-1]
url

'https://kr.noxinfluencer.com/youtube/channel/UCfpaSruWW3S4dibonKXENjA'

In [4]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath("//*[@id=\"tab-channel\"]/div[3]/div[1]/div/span[3]").click()
time.sleep(2)
html = driver.page_source
driver.quit()
soup = BeautifulSoup(html, 'html.parser')

In [5]:
base_data_item = dict(list(zip([(x.get_text()).strip() for x in soup.select('li.base-data-item p.title')],
                               [(x.get_text()).strip() for x in soup.select('li.base-data-item div.value-content span.strong')])))
base_data_item

{'구독자': '406만',
 '조회수': '4.89억',
 '평균 조회수': '277.92만',
 '동영상': '232',
 '라이브 수입': '-'}

In [6]:
# 구독자 랭킹
global_rank = soup.select_one('#global-rank').get_text().strip().split(' ')[0]
area_rank = soup.select_one('#area-rank').get_text().strip().split(' ')[0]
rank = {'global_rank':global_rank,'area_rank':area_rank}
rank

{'global_rank': '3,985th', 'area_rank': '93rd'}

In [7]:
nox_score = soup.select_one('div.noxscore-content span.score').get_text().strip()
nox_score

'4.87'

In [8]:
# 제휴 단가와 월소득
est = soup.select('div.est-content')[0].get_text().replace(" ","").replace('￦','')
monthly_income = {'min' : soup.select('div.est-content')[1].get_text().replace(" ","").replace('￦','').split('-')[0],
                  'max' : soup.select('div.est-content')[1].get_text().replace(" ","").replace('￦','').split('-')[1]}
est, monthly_income

('1.13억(동영상1개당)', {'min': '1.62억', 'max': '2.82억'})

In [9]:
tag_list = [x.get_text() for x in soup.select('li.tag-item a.tag-link')]
tag_list

['Eating show',
 'Food',
 '음식&요리',
 '먹방',
 'MUKBANG',
 'Challenge',
 '大食い',
 'real sound',
 '리얼사운드',
 'social eating',
 '吃播',
 'big eater',
 'Korean mukbang eating show',
 'People & Blogs',
 'ASMR',
 '맛집',
 '라면',
 '고기',
 '치킨',
 '도전먹방']

In [10]:
temp = soup.select('div.table-container table.tabla-ui-content')
table_column = parser.make2d(temp[0])[0]
table_content = parser.make2d(temp[0])[1:]

In [11]:
channel_statistics = []
for x in table_content:
    y = {}    
    for z in range(0,len(table_column)):
        y[table_column[z]] = x[z]
    channel_statistics.append(y)
channel_statistics

[{'날짜': '2021-07-11',
  '구독자': '372만',
  '조회수': '4.1억',
  '유튜브 수익 예측': '-',
  '라이브 수입': '￦ 0',
  '동영상': '+1'},
 {'날짜': '2021-07-12',
  '구독자': '373만 1만',
  '조회수': '4.13억 260.36만',
  '유튜브 수익 예측': '￦ 516.86만 - ￦ 898.88만',
  '라이브 수입': '-',
  '동영상': '-'},
 {'날짜': '2021-07-13',
  '구독자': '374만 1만',
  '조회수': '4.16억 338.67만',
  '유튜브 수익 예측': '￦ 672.32만 - ￦ 1169.26만',
  '라이브 수입': '￦ 0',
  '동영상': '+1'},
 {'날짜': '2021-07-14',
  '구독자': '375만 1만',
  '조회수': '4.19억 298.52만',
  '유튜브 수익 예측': '￦ 592.61만 - ￦ 1030.64만',
  '라이브 수입': '-',
  '동영상': '-'},
 {'날짜': '2021-07-15',
  '구독자': '377만 2만',
  '조회수': '4.23억 375.99만',
  '유튜브 수익 예측': '￦ 746.4만 - ￦ 1298.1만',
  '라이브 수입': '￦ 0',
  '동영상': '+1'},
 {'날짜': '2021-07-16',
  '구독자': '378만 1만',
  '조회수': '4.26억 329.02만',
  '유튜브 수익 예측': '￦ 653.17만 - ￦ 1135.95만',
  '라이브 수입': '-',
  '동영상': '-'},
 {'날짜': '2021-07-17',
  '구독자': '380만 2만',
  '조회수': '4.3억 332만',
  '유튜브 수익 예측': '￦ 659.09만 - ￦ 1146.24만',
  '라이브 수입': '￦ 0',
  '동영상': '+1'},
 {'날짜': '2021-07-18',
  '구독자': '381만 1만',

In [12]:
data = {'name' : name}
data.update(base_data_item)
data['subscriber_rank'] = rank
data['nox_score'] = nox_score
data['est'] = est
data['monthly_income'] = monthly_income
data['tag_list'] = tag_list
data['channel_statistics'] = channel_statistics
data

{'name': '쯔양',
 '구독자': '406만',
 '조회수': '4.89억',
 '평균 조회수': '277.92만',
 '동영상': '232',
 '라이브 수입': '-',
 'subscriber_rank': {'global_rank': '3,985th', 'area_rank': '93rd'},
 'nox_score': '4.87',
 'est': '1.13억(동영상1개당)',
 'monthly_income': {'min': '1.62억', 'max': '2.82억'},
 'tag_list': ['Eating show',
  'Food',
  '음식&요리',
  '먹방',
  'MUKBANG',
  'Challenge',
  '大食い',
  'real sound',
  '리얼사운드',
  'social eating',
  '吃播',
  'big eater',
  'Korean mukbang eating show',
  'People & Blogs',
  'ASMR',
  '맛집',
  '라면',
  '고기',
  '치킨',
  '도전먹방'],
 'channel_statistics': [{'날짜': '2021-07-11',
   '구독자': '372만',
   '조회수': '4.1억',
   '유튜브 수익 예측': '-',
   '라이브 수입': '￦ 0',
   '동영상': '+1'},
  {'날짜': '2021-07-12',
   '구독자': '373만 1만',
   '조회수': '4.13억 260.36만',
   '유튜브 수익 예측': '￦ 516.86만 - ￦ 898.88만',
   '라이브 수입': '-',
   '동영상': '-'},
  {'날짜': '2021-07-13',
   '구독자': '374만 1만',
   '조회수': '4.16억 338.67만',
   '유튜브 수익 예측': '￦ 672.32만 - ￦ 1169.26만',
   '라이브 수입': '￦ 0',
   '동영상': '+1'},
  {'날짜': '2021-07-14',
   

In [13]:
with open("./data/noxinfluencer_{}.json".format(name), 'w',encoding="UTF-8") as outfile:
        json.dump(data, outfile, ensure_ascii = False, indent = 4)